In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Tiny Shakespear
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-24 14:01:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-12-24 14:01:29 (29.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911

MARI he avayokis erceller thour d, myono thishe me tord se by he me, Forder anen: at trselorinjulour t yoru thrd wo ththathy IUShe bavidelanoby man ond be jus as g e atot Meste hrle s, ppat t JLENCOLIUS:
Oppid tes d s o ged moer y pevehear soue maramapay fo t: bueyo malalyo!
Duir.
Fl ke it I t l o'ddre d ondu s?
cr, havetrathackes w.
PUpee meshancun, hrendspouthoulouren whel's'sesoread pe, s whure our heredinsethes; sedsend r lo pamit,
QUMIVIVIOfe m ne RDINid we tr ort; t:
MINENXI l dintandore r


# Mathematical Trick in self-attention models

In [4]:
# considering the following toy example:
torch.manual_seed(1337)
B,T,C= 4,8,2 #batch, time, channels(information at each point in the sequence)
x= torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

##### tokens should not interact with other future tokens, it should only talk to the previous tokens so the information flows from previous context to the current time stamp.


In [5]:
# We want x[b,t]=mean_{i<=t} x[b,i]
#bow means bag of words
xbow= torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev= x[b, :t+1] #(t,C)
        xbow[b,t]= torch.mean(xprev, 0)

In [6]:
wei= torch.tril(torch.ones(T,T))
wei= wei/wei.sum(1, keepdim=True)
xbow2= wei @ x # (B,T,T) @ (B,T,C) ---> (B,T,C)

In [7]:
torch.allclose(xbow, xbow2, atol=1e-6)

True

In [8]:
tril = torch.tril(torch.ones(T,T))
wei= torch.zeros((T,T))
wei= tril.masked_fill(tril == 0, float('-inf'))
wei= F.softmax(wei, dim=-1)
xbow3= wei @ x 
torch.allclose(xbow, xbow3, atol=1e-6)

True

This code computes a **bag-of-words (BoW) style running average** of the input tensor `x`. For each batch index `b` and time step `t`, the value `xbow[b, t]` is set to the mean of all vectors `x[b, i]` where `i ≤ t`. In other words, each position contains the average of the current and all previous token representations. This corresponds to the definition  

$$
x[b,t] = \text{mean}_{   i \le } x[b,i]
$$

and is referred to as a *bag-of-words* representation because it aggregates information from previous tokens without preserving their order beyond inclusion.


In [9]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [10]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

`xbow[0]` is printing with the average of all the past context, <br>Whereas `x[0]` is only considering the present element.

But I don't think any of these is efficient.(#freewill)


In [11]:
torch.manual_seed(42)
a= torch.ones(3,3)
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [12]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [13]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
a= a/torch.sum(a,1, keepdim=True)
b= torch.randint(0,10,(3,2)).float()
c= a@b
print(f'a={a}')
print()
print(f'b= {b}')
print()
print(f'c={c}')

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])

c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [15]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32  # batch, time, channels
x= torch.randn(B,T,C)

tril = torch.tril(torch.ones(T,T))
wei= torch.zeros((T,T))
wei= wei.masked_fill(tril ==0, float('-inf'))
wei= F.softmax(wei, dim=-1)
out= wei@x

out.shape

torch.Size([4, 8, 32])

In [27]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32  # batch, time, channels
x= torch.randn(B,T,C)

# let's see a single head perform self-attention
head_size= 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C,head_size,bias=False)
k= key(x) #(B,T, 16)
q=query(x) #(B,T, 16)
v= value(x)

wei= q @ k.transpose(-2,-1) #(B,T,16)@(B,16,T ) ---> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei= torch.zeros((T,T))
wei= wei.masked_fill(tril ==0, float('-inf'))
wei= F.softmax(wei, dim=-1)
out= wei@v

out.shape

torch.Size([4, 8, 16])

In [25]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [22]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
         [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
         [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
         [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4016, 0.5984, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3365, 0.2271, 0.4364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3019, 0.2060, 0.2899, 0.2022, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1058, 0.1700, 0.1530, 0.3451, 0.2261, 0.0000, 0.0000, 0.0000],
         [0.1526, 0.164

In [26]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

## Version 4: Self-Attention (Single Head)

This code demonstrates a minimal implementation of **single-head causal self-attention**, similar to what is used inside Transformer models.

### Setup
We first fix the random seed for reproducibility and define the input dimensions:
- **B**: batch size  
- **T**: sequence length (time steps / tokens)  
- **C**: embedding dimension (channels)

A random input tensor `x` of shape `(B, T, C)` represents a batch of token embeddings.

### Key, Query, and Value Projections
Self-attention works by projecting the input embeddings into three separate spaces:
- **Keys (K)** determine *what information* each token contains
- **Queries (Q)** determine *what information* each token is looking for
- **Values (V)** contain the information that will be aggregated

Each projection is implemented as a linear layer mapping from `C` to `head_size`.

### Attention Weights
We compute attention scores by taking the dot product between queries and keys:

$$
\text{attention} = QK^\top
$$


This produces a tensor of shape `(B, T, T)`, where each token attends to every other token in the sequence.

### Causal Masking
To prevent tokens from attending to future tokens (important for autoregressive models), a **lower-triangular mask** is applied. Positions above the diagonal are set to `-inf`, ensuring their attention weights become zero after softmax.

### Softmax Normalization
The masked attention scores are passed through a softmax so that each row forms a valid probability distribution over past tokens.

### Weighted Sum of Values
Finally, the attention weights are used to compute a weighted sum of the value vectors:

$$
\text{output} = \text{softmax}(QK^\top) \cdot V
$$

The resulting tensor `out` has shape `(B, T, head_size)` and represents the output of a single self-attention head.


Notes:
* Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
* Each example across batch dimension is of course processed completely independently and never "talk" to each other.
* In an "encoder" attention block just delete the single line that does maskingwith `tril`, allowing all tokens to communicate. This block here is caller a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
* "Self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (eg. an encoder module)
* "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input q,k are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much.

In [28]:
k= torch.randn(B,T, head_size) #(B,T, 16)
q= torch.randn(B,T, head_size) #(B,T, 16)
wei= q@k.transpose(-2,-1) #*head_size**-0.5

In [29]:
k.var()

tensor(1.0449)

In [30]:
q.var()

tensor(1.0700)

In [31]:
wei.var()

tensor(17.4690)

In [37]:
wei= q@k.transpose(-2,-1) *head_size**-0.5

In [38]:
print(k.var())
print(q.var())
print(wei.var())

tensor(1.0449)
tensor(1.0700)
tensor(1.0918)


In [40]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.3,0.5]) ,dim=-1)

tensor([0.1951, 0.1446, 0.2384, 0.1308, 0.2911])

In [42]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.3,0.5])*8 ,dim=-1)

tensor([0.0327, 0.0030, 0.1618, 0.0013, 0.8013])

by multiplying with 8 we just sharepened the variables and it will be wayy too peaky,